In [1]:
# Importing the relevant packages
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins.hparams import api as hp
#hp is hyperparameters
#lets tune kernel size  and optimizer  ( we will tune them)

## Downloading and preprocessing the data

In [2]:

BUFFER_SIZE = 70_000 # same as is
BATCH_SIZE = 128
NUM_EPOCHS = 2

In [3]:
# Downloading the MNIST dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [5]:
# Creating a function to scale our data
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.

    return image, label

In [6]:
# Scaling the data
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [7]:
# size of validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
# Defining size of test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [9]:
# Reshuffle
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
# Splitting the dataset 
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
# Batch
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

## define hyperparameters

In [12]:
# Defining the hypermatarest we would test and their range
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7])) #kernel size (different varitions)
# 3,5 and 7 we will experiment
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# lets experiment the optimizers adam and sgd
METRIC_ACCURACY = 'accuracy'
# you have TEST accuracy and we ARE NOT DISCUSSING IT UNLESS YOU WANT TO FAIL THIS COURSE

# Logging setup info  # yes again log info
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## create your model and training in a function

In [13]:
def train_test_model(hparams):
    # you change hparams[HP_FILTER_SIZE] as 3 5 and 7
    # you change hparams[HP_OPTIMIZER]  for 2 different optimizers
    # first copy paste your CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)
    ])
    
    # Define the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=loss_fn, metrics=['accuracy'])
    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_data, 
        epochs = NUM_EPOCHS,
        callbacks = [early_stopping],
        validation_data = validation_data,
        verbose = 2
    )
    
    _, accuracy = model.evaluate(test_data)
    
    return accuracy

In [14]:
# Creating a function to log the resuls
def run(log_dir, hparams):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

## Creating the model and training it

In [15]:
# Performing a grid search on the hyperparameters we need to test
session_num = 0

for filter_size in HP_FILTER_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:  # oh look now you must use a NESTED LOOP
    
        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_OPTIMIZER: optimizer
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)

        session_num += 1
 # NOW we will need to wait 6 times more because we will train for EACH combination
# you can take a shower, play some games, call your girlfriend/boyfriend, drink a beer, listen some music 

--- Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}
Epoch 1/2
422/422 - 32s - loss: 0.2883 - accuracy: 0.9164 - val_loss: 0.1108 - val_accuracy: 0.9668 - 32s/epoch - 75ms/step
Epoch 2/2
422/422 - 28s - loss: 0.0795 - accuracy: 0.9757 - val_loss: 0.0625 - val_accuracy: 0.9810 - 28s/epoch - 67ms/step
1/1 [==============================] - 1s 1s/step - loss: 0.0548 - accuracy: 0.9823
--- Starting trial: run-1
{'filter_size': 3, 'optimizer': 'sgd'}
Epoch 1/2
422/422 - 28s - loss: 1.3627 - accuracy: 0.6289 - val_loss: 0.4988 - val_accuracy: 0.8480 - 28s/epoch - 67ms/step
Epoch 2/2
422/422 - 30s - loss: 0.4090 - accuracy: 0.8786 - val_loss: 0.3411 - val_accuracy: 0.8993 - 30s/epoch - 70ms/step
1/1 [==============================] - 1s 1s/step - loss: 0.3312 - accuracy: 0.9012
--- Starting trial: run-2
{'filter_size': 5, 'optimizer': 'adam'}
Epoch 1/2
422/422 - 34s - loss: 0.2484 - accuracy: 0.9291 - val_loss: 0.0738 - val_accuracy: 0.9762 - 34s/epoch - 80ms/step
Epoch 2/2
422/4

422/422 - 28s - loss: 0.2444 - accuracy: 0.9305 - val_loss: 0.0760 - val_accuracy: 0.9783 - 28s/epoch - 67ms/step
Epoch 2/2
422/422 - 27s - loss: 0.0718 - accuracy: 0.9786 - val_loss: 0.0536 - val_accuracy: 0.9835 - 27s/epoch - 63ms/step
1/1 [==============================] - 1s 1s/step - loss: 0.0656 - accuracy: 0.9794
--- Starting trial: run-5
{'filter_size': 7, 'optimizer': 'sgd'}
Epoch 1/2


422/422 - 28s - loss: 0.9995 - accuracy: 0.7521 - val_loss: 0.4008 - val_accuracy: 0.8917 - 28s/epoch - 66ms/step
Epoch 2/2
422/422 - 26s - loss: 0.3298 - accuracy: 0.9059 - val_loss: 0.2759 - val_accuracy: 0.9252 - 26s/epoch - 61ms/step
1/1 [==============================] - 1s 976ms/step - loss: 0.2619 - accuracy: 0.9276


## Visualizing in Tensorboard

In [16]:
# Loading the Tensorboard extension   # use it for visualization  #my logs for this training!
# ( do this everytime you run the code below)
%load_ext tensorboard
%tensorboard --logdir "logs/hparam_tuning" 
